### Fullname: Võ Thanh Hiệp
### Student ID: 21127275
### Class: 21CLC03

# 1) Diagonalizable matrix algorithm
## Input a matrix A with n rows and n columns.
 - **Step 1:** Calculate the eigen value
     - **Step 1.1:** Let temp <- A    (copy Matrix)
     - **Step 1.2:** Let temp[i][i] <- temp[i][i] - lambda with 1 <= i <= n
     - **Step 1.3:** Generate the determinant equation from matrix temp
     - **Step 1.4:** Solve the determinant equation, return lambda solutions (set of eigen values)
 - **Step 2:** Let list_Eigen_Vector is empty
 - **Step 3:** With each eigen value,  calculate eigen_vector and append it to list_Eigen_Vector
 - **Step 4:** if the amount of eigen_vector < n, return that 'The matrix A is not diagonalizable.'
 - **Step 5:** Calculate the matrix P, matrix transpose P, matrix D:
     - **Step 5.1:** Generate matrix P from list_Eigen_Vector belong columns
     - **Step 5.2:** Generate matrix transpose P is inversed from matrix P
     - **Step 5.3:** Let matrix D <- transpose P * A
     - **Step 5.4:** Let matrix D <- D * P
 - **Step 6:** return matrix P, matrix transpose P, matrix D

In [1]:
def printMatrix(Matrix):
    nRow = len(Matrix)
    print('[', end="")
    for i in range(nRow - 1):
        print(Matrix[i])
    print(Matrix[nRow - 1], end="")
    print(']')
    
def inputMatrix():
    n = int(input("Enter n (dim of matrix):"))
    
    Matrix = []
    for i in range(n):
        line = []
        for j in range(n):
            cell = int(input("Enter the value in cell [" + str(i + 1) + "][" + str(j + 1) + "]:"))
            line.append(cell)
        Matrix.append(line)
    
    return Matrix

In [2]:
def copyMatrix(Matrix):
    result = []
    nRow = len(Matrix)
    nColumn = len(Matrix[0])
    
    for i in range(nRow):
        row = []
        for j in range(nColumn):
            row.append(Matrix[i][j])
        result.append(row)
            
    return result

In [3]:
def isSquareMatrix(matrix):
    row = len(matrix)
    for i in range(row):
        if row != len(matrix[i]):
            return False
    
    return True

In [4]:
def submatrix(matrix, row, col):
    return [row[:col] + row[col+1:] for row in (matrix[:row] + matrix[row+1:])]


def determinant(matrix):
    n = len(matrix)
    
    if n == 2:
        return matrix[0][0] * matrix[1][1] - matrix[0][1] * matrix[1][0]
    
    det = 0
    for c in range(n):
        det += ((-1) ** c) * matrix[0][c] * determinant(submatrix(matrix, 0, c))
    
    return det

In [5]:
from sympy import Symbol
from sympy.solvers import solve

def calculate_Eigen_Value(matrix):
    n = len(matrix)
    l = Symbol('lambda')
    for i in range(n):
        matrix[i][i] -= l
    
    det = determinant(matrix)
    det = solve(det)
    
    return det

In [6]:
### Reducing all values in row by dividing them by the lead element in that row.

def divideRowWithTheLeadElement(Row, leadPosition, n):
    dividend = Row[leadPosition]
    for i in range(leadPosition, n):
        Row[i] /=  dividend

### If the cell at A[cellRow][cellColumn] = 0. Finding the cell below it that different from value 0.

def findLeadPositionBelowCellInColumn(Matrix, cellRow, cellColumn, nRow):
    for i in range(cellRow + 1, nRow):
        if Matrix[i][cellColumn] != 0:
            return i
    return nRow

"""
When the lead element is found out. Of course, it values will be 1. In the column includes this cell, 
make all cells above and below it to 0.
"""

def clearColumnIncludeLeadElement(Matrix, cellRow, cellColumn, nRow, nColumn):
    for i in range(cellRow + 1, nRow):
        rate = Matrix[i][cellColumn] / Matrix[cellRow][cellColumn]
        for j in range(cellColumn, nColumn):
            Matrix[i][j] -= rate * Matrix[cellRow][j]
    
    for i in range(cellRow):
        rate = Matrix[i][cellColumn] / Matrix[cellRow][cellColumn]
        for j in range(cellColumn, nColumn):
            Matrix[i][j] -= rate * Matrix[cellRow][j]

def Gauss_elimination(Matrix):
    nRow = len(Matrix)
    nColumn = len(Matrix[0])
    
    j = 0
    for i in range(nColumn):
        if Matrix[j][i] == 0:
            rowSwitch = findLeadPositionBelowCellInColumn(Matrix, j, i, nRow)
            if rowSwitch == nRow:
                continue
            Matrix[j], Matrix[rowSwitch] = Matrix[rowSwitch], Matrix[j]
        
        divideRowWithTheLeadElement(Matrix[j], i, nColumn)
        clearColumnIncludeLeadElement(Matrix, j, i, nRow, nColumn)
        
        j += 1
        if j == nRow:
            break
    
    return Matrix

In [7]:
def isNoSolution(Augmented_Matrix, nRow, nColumn):
    for i in reversed(range(nRow)):
        for j in range(nColumn - 1):
            if Augmented_Matrix[i][j] != 0:
                return False
        if Augmented_Matrix[i][nColumn - 1] != 0:
            return True
    return False

def isOneSolution(Augmented_Matrix, nRow, nColumn):
    if nRow < nColumn - 1:
        return False
    for i in range(nRow):
        for j in range(i):
            if Augmented_Matrix[i][j] != 0:
                return False
        for j in range(i + 1, nColumn - 1):
            if Augmented_Matrix[i][j] != 0:
                return False
    return True

def isTheColumnHasFreeVariable(Matrix, theColumn, nRow):
    for i in range(nRow):
        if Matrix[i][theColumn] == 1:
            return False
    return True

def isZeroRow(row):
    length = len(row)
    for i in range(length):
        if row[i] != 0:
            return False;
    return True

def back_substitution(A):
    nRow = len(A)
    nColumn = len(A[0])
    Augmented_Matrix = A
    
    Augmented_Matrix = Gauss_elimination(Augmented_Matrix)
    
    solutions = []
    if isNoSolution(Augmented_Matrix, nRow, nColumn) == True:
        return 'No Solution', solutions
    
    if isOneSolution(Augmented_Matrix, nRow, nColumn) == True:
        while isZeroRow(Augmented_Matrix[nRow - 1]) == True:
            nRow -= 1
        for i in range(nRow):
            solutions.append(Augmented_Matrix[i][nColumn - 1])
        return 'One Solution', solutions
    
    nAscii = 97
    countTemp = 0
    for i in range(nColumn - 1):
        if isTheColumnHasFreeVariable(Augmented_Matrix, i, nRow) == True:
            solutions.append(chr(nAscii))
            nAscii += 1
        else:
            solutions.append('temp')
            countTemp += 1
            
    for i in range(countTemp):
        leadPosition = 0
        print('i:', i)
        print('leadPosition:', leadPosition)
        print(Augmented_Matrix[i])
        while leadPosition < nColumn and Augmented_Matrix[i][leadPosition] != 1:
            leadPosition += 1
            
        solution = str(Augmented_Matrix[i][nColumn - 1])
        for j in range(leadPosition + 1, nColumn - 1):
            if solutions[j] == 'temp':
                continue
            if Augmented_Matrix[i][j] < 0:
                solution += ' + ' + str(-Augmented_Matrix[i][j]) + solutions[j]
            elif Augmented_Matrix[i][j] > 0:
                solution += ' - ' + str(Augmented_Matrix[i][j]) + solutions[j]
        if leadPosition >= len(solutions):
            leadPosition = len(solutions) - 1
        solutions[leadPosition] = solution
    
    return 'Infinitely Many Solutions', solutions

In [8]:
def calculate_Eigen_Vector(matrix, Lambda):
    n = len(matrix)
    for i in range(n):
        matrix[i][i] -= Lambda
        
    matrix = Gauss_elimination(matrix)
    solutions = back_substitution(matrix)
    
    if solutions[0] == 'No Solution':
        position = -1
        for column in range(n):
            count = 0
            for row in range(n):
                if matrix[row][column] == 0:
                    count += 1
                else:
                    break
            if count == n:
                position = column
                break
        
        result = [1 if i == position else 0 for i in range(n)]
        return result
        
    elif solutions[0] == 'One Solution':
        position = -1
        for column in range(n):
            count = 0
            for row in range(n):
                if matrix[row][column] != 0:
                    count += 1
            if count > 1:
                position = column
                break
                
        result = [0] * n
        for column in range(n):
            if column == position:
                continue
            for row in range(n):
                if matrix[row][column] == 0:
                    continue
                result[column] = - matrix[row][position]
                break
                
        result[position] = 1
        return result
    
    return 'no'

In [9]:
def generateMatrixFromSetOfColumnVectors(setVector):
    n = len(setVector)
    Matrix = []
    
    for i in range(n):
        line = []
        for j in range(n):
            temp = setVector[j][i]
            line.append(temp)
        Matrix.append(line)
        
    return Matrix

In [10]:
"""
When the lead element is found out. Of course, it values will be 1. In the column includes this cell, 
make all cells above and below it to 0.
"""
def clearColumnIncludeLeadElement(Matrix, cellRow, cellColumn, nRow, nColumn):
    for i in range(cellRow + 1, nRow):
        rate = Matrix[i][cellColumn] / Matrix[cellRow][cellColumn]
        for j in range(cellColumn, nColumn):
            Matrix[i][j] -= rate * Matrix[cellRow][j]
    
    for i in range(cellRow):
        rate = Matrix[i][cellColumn] / Matrix[cellRow][cellColumn]
        for j in range(cellColumn, nColumn):
            Matrix[i][j] -= rate * Matrix[cellRow][j]
            
# Using for inverse fomulation
def clearColumnOf2MatrixsIncludeLeadElement(Matrix, Augmented_Inverse, cellRow, cellColumn, nRow, nColumn):
    for i in range(cellRow + 1, nRow):
        rate = Matrix[i][cellColumn] / Matrix[cellRow][cellColumn]
        for j in range(cellColumn, nColumn):
            Matrix[i][j] -= rate * Matrix[cellRow][j]
        for j in range(nColumn):
            Augmented_Inverse[i][j] -= rate * Augmented_Inverse[cellRow][j]
    
    for i in range(cellRow):
        rate = Matrix[i][cellColumn] / Matrix[cellRow][cellColumn]
        for j in range(cellColumn, nColumn):
            Matrix[i][j] -= rate * Matrix[cellRow][j]
        for j in range(nColumn):
            Augmented_Inverse[i][j] -= rate * Augmented_Inverse[cellRow][j]

### Reducing all values in row by dividing them by the lead element in that row.
def divideRowWithANumber(Row, dividend, n):
    for i in range(n):
        Row[i] /=  dividend

### If the cell at A[cellRow][cellColumn] = 0. Finding the cell below it that different from value 0.
def findLeadPositionBelowCellInColumn(Matrix, cellRow, cellColumn, nRow):
    for i in range(cellRow + 1, nRow):
        if Matrix[i][cellColumn] != 0:
            return i
    return nRow

def checkIfTheMatrixHasInverse(Matrix, nRow, nColumn):
    if nRow < nColumn:
        return False
    
    for i in range(nColumn):
        if Matrix[i][i] != 1:
            return False
        
    for i in range(nColumn):
        for j in range(nColumn):
            if i == j:
                continue
            if Matrix[i][j] != 0:
                return False
            
    for i in range(nColumn, nRow):
        for j in range(nColumn):
            if Matrix[i][j] != 0:
                return False
            
    return True

def inverse(A):
    Matrix = copyMatrix(A)
    nRow = len(Matrix)
    nColumn = len(Matrix[0])
    
    result_Matrix = []
    for i in range(nRow):
        result_Matrix.append([0] * nColumn)
        
    for i in range(nRow):
        result_Matrix[i][i] = 1
    
    j = 0
    for i in range(nColumn):
        if Matrix[j][i] == 0:
            rowSwitch = findLeadPositionBelowCellInColumn(Matrix, j, i, nRow)
            if rowSwitch == nRow:
                continue
            Matrix[j], Matrix[rowSwitch] = Matrix[rowSwitch], Matrix[j]
            result_Matrix[j], result_Matrix[rowSwitch] = result_Matrix[rowSwitch], result_Matrix[j]
        
        leadElement = Matrix[j][i]
        divideRowWithANumber(Matrix[j], leadElement, nColumn)
        divideRowWithANumber(result_Matrix[j], leadElement, nColumn)
        clearColumnOf2MatrixsIncludeLeadElement(Matrix, result_Matrix, j, i, nRow, nColumn)
        
        j += 1
        if j == nRow:
            break
            
    if checkIfTheMatrixHasInverse(Matrix, nRow, nColumn) == False:
        return 'Ma trận không khả nghịch'
    
    return result_Matrix

In [11]:
def dotProduct(fVector, sVector):
    if len(fVector) != len(sVector):
        return "Cannot dot"
    
    result = 0
    for i in range(len(fVector)):
        result += fVector[i] * sVector[i]
        
    return result

def getColumn(Matrix, pColumn):
    if pColumn > len(Matrix[0]):
        return 'Invalid! Column position is not exists'
    
    row = len(Matrix)
    result = []
    for i in range(row):
        result.append(Matrix[i][pColumn])
        
    return result

def isValidToMultiple(A, B):
    columnA = len(A[0])
    rowB = len(B)
    
    if columnA == rowB:
        return True
    return False

def multiple2Matrixs(A, B):
    if isValidToMultiple(A, B) == False:
        return 'Invalid! Cannot multiple'
    
    result = []
    row = len(A)
    column = len(B[0])
    
    for i in range(row):
        line = []
        for j in range(column):
            columnB = getColumn(B, j)
            line.append(dotProduct(A[i], columnB))
        result.append(line)
        
    return result

In [12]:
def Diagonalizable_Matrix(matrix):
    if isSquareMatrix(matrix) == False:
        return "The matrix is not diagonalizable.", 0, 0
    
    eigen_value = calculate_Eigen_Value(copyMatrix(matrix))
    list_Eigen_Vector = []
    nLambda = len(eigen_value)
    
    for i in range(nLambda):
        eigen_vector = calculate_Eigen_Vector(copyMatrix(matrix), eigen_value[i])
        list_Eigen_Vector.append(eigen_vector)
        
    if len(list_Eigen_Vector) < len(matrix):
        return "The matrix is not diagonalizable.", 0, 0
    
    P = generateMatrixFromSetOfColumnVectors(list_Eigen_Vector)
    transposeP = inverse(P)
    D = multiple2Matrixs(transposeP, matrix)
    D = multiple2Matrixs(D, P)
    
    return P, transposeP, D

In [13]:
A = inputMatrix()

Enter n (dim of matrix):2
Enter the value in cell [1][1]:-1
Enter the value in cell [1][2]:3
Enter the value in cell [2][1]:-2
Enter the value in cell [2][2]:4


In [14]:
print("Matrix A:")
printMatrix(A)
print()

P, transposeP, D = Diagonalizable_Matrix(A)

if P == "The matrix is not diagonalizable.":
    print(P)
else:
    print("Matrix P:")
    printMatrix(P)
    print()

    print("Matrix D:")
    printMatrix(D)
    print()
    
    print("Transpose matrix P:")
    printMatrix(transposeP)

Matrix A:
[[-1, 3]
[-2, 4]]

Matrix P:
[[3/2, 1]
[1, 1]]

Matrix D:
[[1, 0]
[0, 2]]

Transpose matrix P:
[[2, -2]
[-2, 3]]


##  Functions describe:
 1. **inputMatrix:** input matrix from key board
 2. **printMatrix:** print matrix 
 3. **copyMatrix:** copy new matrix by argument matrix
 4. **isSquareMatrix:** check if the argument matrix is square matrix meaning number of columns and number of row are equal
 5. **submatrix:** get submatrix from argument matix except a column and a row
 6. **determinant:** find the determinant equation
 7. **calculate_Eigen_Value:** calculate determinant solutions (eigen values) 
 8. **Gauss_elimination:** calculate the matrix by Gauss elimination
 9. **back_substitution:** find all solutions of the argument matrix
 10. **calculate_Eigen_Vector:** find all eigen vectors from an argument matrix and a lambda value
 11. **generateMatrixFromSetOfColumnVectors:** generate matrix from set of column vectors
 12. **inverse:** generate inverse matrix from argument matrix
 13. **dotProduct:** calculate dot product of 2 vectors
 14. **getColumn:** return a vector by column at the position pColumn in Matrix
 15. **isValidToMultiple:** if column of matrix A is equal row of matrix B, return True. Otherwise, return False.
 16. **multiple2Matrixs:** multiple matrix A by matrix B

# 2) EXTEND:

In [15]:
import numpy as np

def find_diagonalization(A):
    # Tìm ma trận chéo hóa P và ma trận đường chéo D
    eigenvalues, eigenvectors = np.linalg.eig(A)
    
    # Ma trận chéo P (eigenvectors) và ma trận đường chéo D (eigenvalues)
    P = eigenvectors
    D = np.diag(eigenvalues)
    
    # Ma trận nghịch đảo của P
    P_inv = np.linalg.inv(P)
    
    return P, D, P_inv

# Ma trận A có thể chéo hóa được (Input ở trên)

# Tìm ma trận chéo P, ma trận đường chéo D và ma trận nghịch đảo của P
P, D, P_inv = find_diagonalization(A)

print("Ma trix:")
printMatrix(A)
print()

print("Ma trận chéo P:")
print(P)

print("\nMa trận đường chéo D:")
print(D)

print("\nMa trận nghịch đảo của P:")
print(P_inv)

Ma trix:
[[-1, 3]
[-2, 4]]

Ma trận chéo P:
[[-0.83205029 -0.70710678]
 [-0.5547002  -0.70710678]]

Ma trận đường chéo D:
[[1. 0.]
 [0. 2.]]

Ma trận nghịch đảo của P:
[[-3.60555128  3.60555128]
 [ 2.82842712 -4.24264069]]


# Comparing results
### The results of algorithm in Topic 1 and numpy in Topic 2 (extend):
 - The bolt diagonalizable matrix D results are the same
 - The diagonalizable matrix P results are not the same
 - The inversed matrix P results are not the same

#### Reason of the different above: The numpy's algorithm uses different eigen vector

## Diagonalizable matrix applications


1. **Eigenvalue Problems:** Diagonalizable matrices are particularly useful when solving eigenvalue problems. Finding eigenvalues and eigenvectors of a diagonalizable matrix is straightforward since the diagonal form of the matrix makes the eigenvalues easily identifiable along the diagonal, and the eigenvectors are simply the columns of the matrix of eigenvectors.

2. **Differential Equations:** In many physical and engineering systems, differential equations arise to describe the behavior of the system. Diagonalizable matrices play a vital role in solving systems of ordinary differential equations and partial differential equations. The solutions can often be represented in terms of exponential functions of diagonal matrices, which significantly simplifies the analysis.

3. **Linear Transformations:** In linear algebra, matrices represent linear transformations. Diagonalizable matrices are helpful when studying linear transformations because they provide a clear picture of how the transformation scales along independent directions represented by the eigenvectors.

4. **Markov Chains:** Diagonalizable matrices are used in the study of Markov chains, which are stochastic processes that transition between states with certain probabilities. The diagonal form of a diagonalizable matrix allows for the computation of long-term behavior and steady-state probabilities of the Markov chain.

5. **Normal Modes in Vibrations:** In physics and engineering, diagonalizable matrices are employed to study vibrations and oscillations in mechanical systems. The diagonalization process reveals the normal modes of vibration, which are essential in analyzing the behavior of complex systems.

6. **Quantum Mechanics:** In quantum mechanics, operators representing physical observables (e.g., position, momentum, energy) are often represented by matrices. Diagonalizing these matrices allows us to find the eigenvalues (observable values) and the corresponding eigenvectors (states) of the quantum system.

7. **Principal Component Analysis (PCA):** PCA is a statistical technique used for dimensionality reduction and data analysis. It involves finding the eigenvectors and eigenvalues of the covariance matrix, which is often a diagonalizable matrix. The eigenvectors correspond to the principal components, and the eigenvalues indicate their importance in explaining the data variance.

8. **Control Theory:** In control theory, diagonalizable matrices are used to analyze the stability and controllability of dynamic systems. The eigenvalues of the diagonalized matrix provide insights into the system's stability and response to control inputs.

9. **Graph Theory:** Some graph-related problems involve the adjacency matrix of a graph, which can often be diagonalized. This helps in understanding various properties of graphs and analyzing network structures.

These are just a few examples of the many applications of diagonalizable matrices. Diagonalization is a powerful tool that simplifies the analysis of matrices and provides a deeper understanding of the systems they represent.